# Modelos: LLM

In [44]:
%load_ext autoreload
%autoreload 2

In [45]:
BASE_PATH             = '../..'
LIB_PATH              = f'{BASE_PATH}/lib'
DATASET_PATH          = f'{BASE_PATH}/datasets'

In [46]:
import sys
sys.path.append(LIB_PATH)

import torch
from torch.nn import Sigmoid, ReLU
from torch.optim import SparseAdam, Adam
from torch.utils.data import Dataset, DataLoader

from torchmetrics import R2Score
from torch.nn import MSELoss

import pytorch_common.util as pu
from pytorch_common.modules.fn import Fn
from pytorch_common.callbacks import EarlyStop, \
                                     ReduceLROnPlateau, \
                                     Validation, \
                                     SaveBestModel
from pytorch_common.callbacks.output import Logger, \
                                            MetricsPlotter


import data as dt
import data.dataset as ds

import metric as mt
import metric.discretizer as dr

import numpy as np
import pandas as pd

import model as ml
import data.plot as pl

import util as ut


from rest.ollama import OllamaApiClient

2023-12-15 21:47:12.033803: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-15 21:47:12.614355: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-12-15 21:47:13.210518: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-15 21:47:13.220865: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GP

<Figure size 640x480 with 0 Axes>

## Setup

In [3]:
pu.LoggerBuilder().on_console().build()

<RootLogger root (INFO)>

In [4]:
import pytorch_common
pytorch_common.__version__

'0.3.8'

In [5]:
torch.__version__

'2.1.0+cu118'

In [6]:
pu.set_device_name('gpu')

pu.get_device(), torch.cuda.is_available()

(device(type='cuda', index=0), True)

## Model

In [71]:
def query(msg, model='default'):
    return OllamaApiClient(model=model).query(msg)

query("¿Por que el cielos es azul?")

Query:
¿Por que el cielos es azul?

Response:
¡Hola! *adjusts sunglasses* The sky is blue because of a phenomenon called Rayleigh scattering. When sunlight enters the Earth's atmosphere, it encounters tiny molecules of gases like nitrogen and oxygen. These molecules scatter the light in all directions, but they scatter shorter (blue) wavelengths more than longer (red) wavelengths. This is why the sky appears blue to our eyes! *winks* ¿Quieres más información sobre este tema? 😊

In [72]:
query("Contame un chiste")

Query:
Contame un chiste

Response:¡Haha, ¡qué buena oportunidad! Aquí tienes un chiste para ti:

¿Por qué el abeja se metió en la cocina? ¡Para hacer una torta! 😂

Espero que te haya gustado. ¿Quieres más?